imports

In [1]:
from tqdm.notebook import tqdm as tqdm
import os
import json
import pandas as pd

from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
from myhelpers import TrialStatistics

parameters

In [2]:
experimentsPath="/raid/elhamod/Fish/experiments/"
dataPath="/raid/elhamod/Fish/"
experimentName="biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy"

cuda=0

file and changes

In [3]:
old_params_file = os.path.join(experimentsPath, experimentName, "params.json")

# Keep same order as ConfigParserWriter!
changes = {
    'phylogeny_loss_epsilon': 0.1
}
removes = []

prepare

In [4]:
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)

# populate a hash table of all experiments, indexed by trial hash
experiments = {}
for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
    num_of_models = experiment_params["numOfTrials"]
    for k in range(num_of_models):
        old_trialHash = TrialStatistics.getTrialName(experiment_params, k)
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        new_trialHash = TrialStatistics.getTrialName(experiment_params_new, k)
        experiments[old_trialHash] = {"trialHash": new_trialHash}

Fix models

In [5]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        num_of_models = experiment_params["numOfTrials"]
        with tqdm(total=num_of_models, desc="trial") as bar2:
            for k in range(num_of_models):
            
                # Fix model names
                old_model_name = getModelName(experiment_params, k)
                old_modelPath = os.path.join(experimentsPath, experimentName, old_model_name)
                new_model_name = getModelName(experiment_params_new, k)
                new_modelPath = os.path.join(experimentsPath, experimentName, new_model_name)

                try:
                    if os.path.exists(old_modelPath):
                        os.rename(old_modelPath, new_modelPath)
                        print("model", old_modelPath, '->', new_modelPath, "updated")

                        j = json.dumps(experiment_params_new)
                        json_fileName = os.path.join(new_modelPath, 'params.json')
                        f = open(json_fileName,"w")        
                        f.write(j)
                        f.close()  
                        print("json", json_fileName, "updated")
                    else:
                        raise
                    
                except:
                    print("model", old_model_name, "could not be fixed to", new_model_name)
                    pass  
                
                

                bar2.update()
                
                experiments[TrialStatistics.getTrialName(experiment_params, k)]['modelName'] = new_model_name

        bar.update()

model /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/dc19ceb70a2bb74bd40226e29d2c3836a9661db8c1a6d25412a4ce64 -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/dc19ceb70a2bb74bd40226e29d2c3836a9661db8c1a6d25412a4ce64 updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/dc19ceb70a2bb74bd40226e29d2c3836a9661db8c1a6d25412a4ce64/params.json updated
model /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/f588c02456be120d71feb48d57a2ef911d1e0e924c44dd5c3703113b -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/f588c02456be120d71feb48d57a2ef911d1e0e924c44dd5c3703113b updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/f588c02456be120d71feb48d57a2ef911d1e0e9

model /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/de716be0d590b45554d9a7cb47e01c436bcde97ab69c4ddaf3d82ba9 -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/de716be0d590b45554d9a7cb47e01c436bcde97ab69c4ddaf3d82ba9 updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/de716be0d590b45554d9a7cb47e01c436bcde97ab69c4ddaf3d82ba9/params.json updated
model /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/fc6305989f8ec2bf1b923d6fb4dac77a240833f642bb640a41a2903f -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/fc6305989f8ec2bf1b923d6fb4dac77a240833f642bb640a41a2903f updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/fc6305989f8ec2bf1b923d6fb4dac77a240833f

model /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/6eabb328391911f1bdc4d8504985c5af86b58a0e25efa2f5f1f6f2c0 -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/6eabb328391911f1bdc4d8504985c5af86b58a0e25efa2f5f1f6f2c0 updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/6eabb328391911f1bdc4d8504985c5af86b58a0e25efa2f5f1f6f2c0/params.json updated
model /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/a366936e54c0b23d57f0a01eb37217d6f2c629dc61f9369c10b9ff79 -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/a366936e54c0b23d57f0a01eb37217d6f2c629dc61f9369c10b9ff79 updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/a366936e54c0b23d57f0a01eb37217d6f2c629d

model /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/137943dac98f6086c258e1b3462389fb485d114ef38258f45c11c891 -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/137943dac98f6086c258e1b3462389fb485d114ef38258f45c11c891 updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/137943dac98f6086c258e1b3462389fb485d114ef38258f45c11c891/params.json updated
model /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/eb775da610d55e5749a2764fbf6f9292679d289c8601ea737ea81a03 -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/eb775da610d55e5749a2764fbf6f9292679d289c8601ea737ea81a03 updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/eb775da610d55e5749a2764fbf6f9292679d289

model /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/bdff04a1e88bc8806a77af00b141824b996a2e00ea223e9e4ee409de -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/bdff04a1e88bc8806a77af00b141824b996a2e00ea223e9e4ee409de updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/bdff04a1e88bc8806a77af00b141824b996a2e00ea223e9e4ee409de/params.json updated
model /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/c03bf839740a2db3766400ade9a644916be1aa4312d7b420d07ea015 -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/c03bf839740a2db3766400ade9a644916be1aa4312d7b420d07ea015 updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/c03bf839740a2db3766400ade9a644916be1aa4

model /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/64942411a4fd817835a08a65ab91a33b28e4166652ac63aeeb2d6ac0 -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/64942411a4fd817835a08a65ab91a33b28e4166652ac63aeeb2d6ac0 updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/64942411a4fd817835a08a65ab91a33b28e4166652ac63aeeb2d6ac0/params.json updated
model /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/d23d750536efe8b9a3d773629cb01d3e03d3f6faa465192afd3a0430 -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/d23d750536efe8b9a3d773629cb01d3e03d3f6faa465192afd3a0430 updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/d23d750536efe8b9a3d773629cb01d3e03d3f6f

model /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/1d18b584208fd7410b2f90d9ce619f224d8bdf22c0c2001a18e51bb9 -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/1d18b584208fd7410b2f90d9ce619f224d8bdf22c0c2001a18e51bb9 updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/1d18b584208fd7410b2f90d9ce619f224d8bdf22c0c2001a18e51bb9/params.json updated
model /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/d9279fae08f4d96edcedb1fdbf6772396793c7d11081b1dfda1f38e7 -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/d9279fae08f4d96edcedb1fdbf6772396793c7d11081b1dfda1f38e7 updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/d9279fae08f4d96edcedb1fdbf6772396793c7d

model /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/4c79204c6333dd10a48f116c9d0beabdb58e48bea8ab7512cfc40c9d -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/4c79204c6333dd10a48f116c9d0beabdb58e48bea8ab7512cfc40c9d updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/4c79204c6333dd10a48f116c9d0beabdb58e48bea8ab7512cfc40c9d/params.json updated
model /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/38fcb94fbbb2c23d97c33598f8270f1df43a13d7bb1661ef424e3208 -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/38fcb94fbbb2c23d97c33598f8270f1df43a13d7bb1661ef424e3208 updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/38fcb94fbbb2c23d97c33598f8270f1df43a13d

model /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/7fe07f7bfe3cc12cddee63e76c89fb4a8b712c3adc516bf7b65f53fc -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/7fe07f7bfe3cc12cddee63e76c89fb4a8b712c3adc516bf7b65f53fc updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/7fe07f7bfe3cc12cddee63e76c89fb4a8b712c3adc516bf7b65f53fc/params.json updated
model /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/6e63ba238f0c345b8435cc42acfd2cf9ffaeaf2fa720222f13d669b5 -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/6e63ba238f0c345b8435cc42acfd2cf9ffaeaf2fa720222f13d669b5 updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/6e63ba238f0c345b8435cc42acfd2cf9ffaeaf2

model /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/3f52d952b5bfb8b7e9874d0c556ad05dbfffce9d6b34ed84e63f0f29 -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/3f52d952b5bfb8b7e9874d0c556ad05dbfffce9d6b34ed84e63f0f29 updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/3f52d952b5bfb8b7e9874d0c556ad05dbfffce9d6b34ed84e63f0f29/params.json updated
model /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/2a0a6affecaffea3933c6e015fd87a369e538fd808dd841b4dfa7e85 -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/2a0a6affecaffea3933c6e015fd87a369e538fd808dd841b4dfa7e85 updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/models/2a0a6affecaffea3933c6e015fd87a369e538fd

Fix results directory

In [6]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        old_experimentHash = TrialStatistics.getTrialName(experiment_params)
        old_experimentPath = os.path.join(experimentsPath, experimentName, "results", old_experimentHash)
        new_experimentHash = TrialStatistics.getTrialName(experiment_params_new)
        new_experimentPath = os.path.join(experimentsPath, experimentName, "results", new_experimentHash)
        
        try:
            if os.path.exists(old_experimentPath):

                os.rename(old_experimentPath, new_experimentPath)
                print("result", old_experimentPath, '->', new_experimentPath, "updated")

                j = json.dumps(experiment_params_new)
                json_fileName = os.path.join(new_experimentPath, 'params.json')
                f = open(json_fileName,"w")        
                f.write(j)
                f.close()  
                print("json", json_fileName, "updated")
            else:
                raise
                
        except:
            print("result", old_experimentHash, "could not be fixed to",new_experimentHash)
            pass 
        
        experiments[TrialStatistics.getTrialName(experiment_params, 0)]['experimentHash'] = new_experimentHash
        
        bar.update()

result /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/results/cd954191236a3eb6d93e159060df32b8bb8e3b5401aa6213fdc26230 -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/results/cd954191236a3eb6d93e159060df32b8bb8e3b5401aa6213fdc26230 updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/results/cd954191236a3eb6d93e159060df32b8bb8e3b5401aa6213fdc26230/params.json updated
result /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/results/e1b89f3118e5d9901ff722c3a84f23b15a4ffc74795db8acbd70a089 -> /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/results/e1b89f3118e5d9901ff722c3a84f23b15a4ffc74795db8acbd70a089 updated
json /raid/elhamod/Fish/experiments/biology_paper_curated4_Easy50_tryphylogeny_weightedCrossEntropy/results/e1b89f3118e5d9901ff722c3a84f23b

Fix experiments file

In [7]:
# open experiments.csv file
experimentsFileNameAndPath=os.path.join(experimentsPath, "experiments.csv")
experiments_file= pd.read_csv(experimentsFileNameAndPath)

# For all experiments
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        numOfTrials = experiment_params["numOfTrials"]
        trialHash_0 = TrialStatistics.getTrialName(experiment_params, 0) # used for keys who don't change for different trials
        for key in removes:
            experiment_params_new.pop(key, None)
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        
        # For all trials
        with tqdm(total=num_of_models, desc="trial") as bar2:
            for j in range(numOfTrials):
                
                trialHash = TrialStatistics.getTrialName(experiment_params, j)
                try:
                    # If old entry doesn't exist, nothing to update
                    record_exists = not (experiments_file[experiments_file['trialHash'] == trialHash][experiments_file['experimentName'] == experimentName]).empty
                    if record_exists:
                        
                        # drop old row and caluclate new row
                        new_trialHash = experiments[trialHash]["trialHash"]
                        experiments_file.drop(experiments_file[experiments_file['trialHash'] == trialHash][experiments_file['experimentName'] == experimentName].index, inplace = True) 
                        row_information = {
                            'experimentName': experimentName,
                            'modelName': experiments[trialHash]["modelName"],
                            'datasetName': experiments[trialHash_0]["datasetName"],
                            'experimentHash': experiments[trialHash_0]["experimentHash"],
                            'trialHash': new_trialHash
                        }
                        row_information = {**row_information, **experiment_params_new} 

                        # error handling to avoid duplicates
                        record_exists = not (experiments_file[experiments_file['trialHash'] == new_trialHash][experiments_file['experimentName'] == experimentName]).empty
                        if record_exists:
                            experiments_file.drop(experiments_file[experiments_file['trialHash'] == new_trialHash][experiments_file['experimentName'] == experimentName].index, inplace = True) 

                        # update with new entry    
                        experiments_file = experiments_file.append(pd.DataFrame(row_information, index=[0]), ignore_index = True)
                        print("trialHash", trialHash, "->", new_trialHash, "updated")
                except:
                    print("trialHash", trialHash, "could not be updated!")
                    pass
                bar2.update()
        bar.update()

# resave file
experiments_file.to_csv(experimentsFileNameAndPath, header=True, index=False)

/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Fix params file

In [8]:
experiment_list = []
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
        print(experiment_params_new)
        experiment_list.append(experiment_params_new)
        bar.update()


j = json.dumps({"experimentList": experiment_list})
f = open(old_params_file,"w")        
f.write(j)
f.close()  
print("json", old_params_file, "updated")

{'image_path': 'Curated4/Easy_50', 'suffix': None, 'img_res': 448, 'augmented': True, 'batchSize': 64, 'learning_rate': 0.0001, 'numOfTrials': 3, 'fc_layers': 1, 'modelType': 'BB', 'lambda': 0.01, 'unsupervisedOnTest': False, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.015, 'adaptive_alpha': 0.8, 'noSpeciesBackprop': False, 'phylogeny_loss': True, 'phylogeny_loss_epsilon': 0.1}
{'image_path': 'Curated4/Easy_50', 'suffix': None, 'img_res': 448, 'augmented': True, 'batchSize': 128, 'learning_rate': 0.0001, 'numOfTrials': 3, 'fc_layers': 1, 'modelType': 'BB', 'lambda': 0.01, 'unsupervisedOnTest': False, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.015, 'adaptive_alpha': 0.8, 'noSpeciesBackprop': False, 'phylogeny_loss': True, 'phylogeny_loss_epsilon': 0.1}
{'image_path': 'Curated4/Easy_50', 'suffix': None, 'img_res': 448, 'augmented': True, 'batchSize': 64, 'learning_rate': 5e-05, 'n